In [37]:
import shots_data_retriever
import importlib

importlib.reload(shots_data_retriever)

<module 'shots_data_retriever' from 'c:\\Users\\Dell.DESKTOP-4AU2O53\\Documents\\UdeM\\IFT 6758\\Projet\\ift6758\\visualizations\\shots_data_retriever.py'>

In [38]:
shotsDataRetriever = shots_data_retriever.ShotsDataRetriever()

df = shotsDataRetriever.get_df_for_milestone2_part4()

print(df)

       period time_in  is_goal  x_coord  y_coord shot_type  is_empty_net  \
0           1   01:11        0     77.0      5.0     wrist             0   
1           1   02:53        0     86.0     13.0     wrist             0   
2           1   04:01        0     23.0    -38.0     wrist             0   
3           1   04:46        0     33.0    -15.0      slap             0   
4           1   06:46        0     34.0     28.0     wrist             0   
...       ...     ...      ...      ...      ...       ...           ...   
68495       3   16:45        0     77.0    -23.0  backhand             0   
68496       3   17:20        0     76.0     38.0      slap             0   
68497       3   17:22        0     79.0     10.0  backhand             0   
68498       3   18:50        0     81.0     -7.0      snap             0   
68499       3   19:39        0     67.0    -14.0      snap             0   

        distance  angle_to_goal  time_in_seconds  game_seconds  
0      13.928388      

In [ ]:
print(df['time_in_seconds'].dtype)
print(df['period'].unique())
print(df[df['period'].isin([4,5])])

int64
[1 2 3 4 5]
       period time_in  is_goal  x_coord  y_coord shot_type  is_empty_net  \
67          4   00:37        1     49.0      5.0      snap             0   
428         4   00:24        0     66.0    -11.0     wrist             0   
429         4   00:51        0     81.0     -7.0     wrist             0   
430         4   01:30        0     79.0     10.0     wrist             0   
431         4   01:39        0     73.0     -7.0     wrist             0   
...       ...     ...      ...      ...      ...       ...           ...   
68171       5   00:00        1     70.0     -6.0     wrist             0   
68172       5   00:00        0     78.0      5.0  backhand             0   
68173       5   00:00        0     66.0      0.0      snap             0   
68433       4   01:02        0     63.0     -9.0     wrist             0   
68434       4   02:50        1     72.0     -3.0    tip-in             0   

        distance  angle_to_goal  time_in_seconds  game_seconds  
67  